In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

llm = ChatGroq(model = "qwen-2.5-32b")

result = llm.invoke("Hello")
result

In [ ]:
from typing import List 
from typing_extensions import TypedDict
from pydantic import BaseModel
from langchain_core.pydantic_v1 import Field

class Analyst(BaseModel):
    affiliation: str = Field(
        description = "primary affiliation of analyst"
    )
    name:str = Field(
        description = "Name of the Analyst"
    )
    role:str = Field(
        description = "Role of the analyst in the context of topic"
    )
    description:str = Field(
        description="Description of the analyst focus,concerns and motives"
    )
    @property
    def persona(self)->str:
        return f"Name: {self.name}\n Role: {self.role}\n Affiliation: {self.affiliation}\n Description: {self.description}\n"
    
class Perspectives(BaseModel):
    analysts: List[Analyst] = Field(
        description = "Comprehensive list of analysts with their roles and affiliations"
    )

class GenerateAnalystsState(TypedDict):
    topic:str
    max_analysts:str
    human_analyst_feedback:str
    analysts: List[Analyst]

In [ ]:
from IPython.display import Image,display
from langgraph.graph import StateGraph,START,END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

analyst_instructions = """ 

  You are tasked with creating a set of AI analyst personas. Follow these instructions carefully:

  1. First, review the research topic:
     {topic}

  2. Examine any editorial feedback that has been optionally provided to guide creation of analysts:
     {human_analyst_feedback}

  3. Determine the most interesting themes based up on documents and/or feedback above.
     
  4. Pick the top {max_analysts} themes.

  5. Assign one analyst to each theme.

"""

In [ ]:
def create_analysts(state:GenerateAnalystsState):
    """create analysts"""

    topic = state["topic"]
    max_analysts = state["max_analysts"]
    human_analyst_feedback = state.get("human_analyst_feedback","")

    # Enforce structured output
    structured_llm = llm.with_structured_output(Perspectives)

    # system message
    system_message = analyst_instructions.format(topic=topic,
                                                 max_analysts=max_analysts,
                                                 human_analyst_feedback=human_analyst_feedback)
    
    analysts = structured_llm.invoke([SystemMessage(content=system_message)]+[HumanMessage(content = "Generate the set of analysts")])

    return {"analysts": analysts.analysts}

In [ ]:
def human_feedback(state:GenerateAnalystsState):
    """No-op node that should be interrupted on"""
    pass

In [ ]:
def should_continue(state:GenerateAnalystsState):
    """Return the next node to execute"""

    #check if human feedback
    human_analyst_feedback = state.get("human_analyst_feedback",None)

    if human_analyst_feedback:
        return "create_analysts"
    
    return END

In [ ]:
# Add nodes and edges
builder = StateGraph(GenerateAnalystsState)

builder.add_node("create_analysts",create_analysts)
builder.add_node("human_feedback",human_feedback)
builder.add_edge(START,"create_analysts")
builder.add_edge("create_analysts","human_feedback")
builder.add_conditional_edges("human_feedback",
                              should_continue,
                              ["create_analysts",END]
                              )
memory = MemorySaver()

graph = builder.compile(checkpointer=memory,interrupt_before=["human_feedback"])

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Input
max_analysts = 3
topic = "The benefits of adopting LangGraph as an agent framework"
thread = {"configurable":{"thread_id":1}}

In [ ]:
for event in graph.stream({"topic":topic,"max_analysts":max_analysts},thread,stream_mode="values"):
    analysts = event.get("analysts","")
    if analysts:
        for analyst in analysts:
            print(f"Name: {analyst.name}")
            print(f"Affiliation: {analyst.affiliation}")
            print(f"Role: {analyst.role}")
            print(f"Description: {analyst.description}")
            print("-"*50)

In [ ]:
state = graph.get_state(thread)
state.next

In [ ]:
graph.update_state(thread,{"human_analyst_feedback":"Add in someone from a startup to add an enterpreneur perspective"},as_node="human_feedback")

In [ ]:
for event in graph.stream(None,thread,stream_mode="values"):
    analysts = event.get("analysts","")
    if analysts:
        for analyst in analysts:
            print(f"Name: {analyst.name}")
            print(f"Affiliation: {analyst.affiliation}")
            print(f"Role: {analyst.role}")
            print(f"Description: {analyst.description}")
            print("-"*50)

In [ ]:
further_feedback = None 
graph.update_state(thread,{"human_analyst_feedback":further_feedback},as_node="human_feedback")

In [ ]:
for event in graph.stream(None,thread,stream_mode="updates"):
    print("--Node--")
    node_name = next(iter(event.keys()))
    print(node_name)

In [ ]:
final_state = graph.get_state(thread)
analysts = final_state.values.get("analysts")

In [ ]:
analysts